# Environment Setup and Data Loading

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from google.cloud import storage


In [ ]:
food_names = [
    "apple_pie", "baby_back_ribs", "baklava", "beef_carpaccio", "beef_tartare", "beet_salad",
    "beignets", "bibimbap", "bread_pudding", "breakfast_burrito", "bruschetta", "caesar_salad",
    "cannoli", "caprese_salad", "carrot_cake", "ceviche", "cheesecake", "cheese_plate",
    "chicken_curry", "chicken_quesadilla", "chicken_wings", "chocolate_cake", "chocolate_mousse",
    "churros", "clam_chowder", "club_sandwich", "crab_cakes", "creme_brulee", "croque_madame",
    "cup_cakes", "deviled_eggs", "donuts", "dumplings", "edamame", "eggs_benedict", "escargots",
    "falafel", "filet_mignon", "fish_and_chips", "foie_gras", "french_fries", "french_onion_soup",
    "french_toast", "fried_calamari", "fried_rice", "frozen_yogurt", "garlic_bread", "gnocchi",
    "greek_salad", "grilled_cheese_sandwich", "grilled_salmon", "guacamole", "gyoza", "hamburger",
    "hot_and_sour_soup", "hot_dog", "huevos_rancheros", "hummus", "ice_cream", "lasagna",
    "lobster_bisque", "lobster_roll_sandwich", "macaroni_and_cheese", "macarons", "miso_soup",
    "mussels", "nachos", "omelette", "onion_rings", "oysters", "pad_thai", "paella", "pancakes",
    "panna_cotta", "peking_duck", "pho", "pizza", "pork_chop", "poutine", "prime_rib",
    "pulled_pork_sandwich", "ramen", "ravioli", "red_velvet_cake", "risotto", "samosa", "sashimi",
    "scallops", "seaweed_salad", "shrimp_and_grits", "spaghetti_bolognese", "spaghetti_carbonara",
    "spring_rolls", "steak", "strawberry_shortcake", "sushi", "tacos", "takoyaki", "tiramisu",
    "tuna_tartare", "waffles"
]

In [ ]:
food_dirs = {food: os.path.join('data', food) for food in food_names}

# Base directory for data
base_dir = 'data'

In [ ]:
data = []

# Iterate over each food category and gather image paths
for food in food_names:
    food_dir = os.path.join(base_dir, food)
    if os.path.exists(food_dir):
        for img_file in os.listdir(food_dir):
            img_path = os.path.join(food_dir, img_file)
            data.append((img_path, food))
    else:
        print(f"{food}: Directory not found at {food_dir}")

# Convert the data into a DataFrame for easier handling (optional)
df = pd.DataFrame(data, columns=['image_path', 'label'])

In [ ]:
df

# Data Splitting (Train, Validation, Test)

In [ ]:
# Splitting into train (85%) and temp (15%)
train_df, temp_df = train_test_split(df, train_size=0.85, random_state=1)

# Further split temp into validation (7.5%) and test (7.5%)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=1)


# Data Augmentation and ImageDataGenerator Setup


In [ ]:
train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# ImageDataGenerator for validation and test data (no augmentation)
test_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input
)

In [ ]:
train_images = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='image_path',
    y_col='label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42
)

# Flow from DataFrame for validation images
val_images = test_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='image_path',
    y_col='label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)

# Flow from DataFrame for test images
test_images = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='image_path',
    y_col='label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)

# Feature Extraction and Model Definition

In [ ]:
def feature_extractor(inputs):
    feature_extractor = EfficientNetB7(input_shape=(224, 224, 3), include_top=False, weights='imagenet')(inputs)
    return feature_extractor

In [ ]:
def classifier(inputs):
    x = GlobalAveragePooling2D()(inputs)
    x = Flatten()(x)
    x = Dense(1024, activation="relu")(x)
    x = Dense(512, activation="relu")(x)
    x = Dropout(0.2)(x)
    x = Dense(101, activation="softmax", name="classification")(x)
    return x

In [ ]:
def define_compile_model():
    inputs = Input(shape=(224, 224, 3))
    efficientnet_feature_extractor = feature_extractor(inputs)
    classification_output = classifier(efficientnet_feature_extractor)
    model = Model(inputs=inputs, outputs=classification_output)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
model = define_compile_model()

In [ ]:
print(model.summary())

 # Model Training

In [ ]:
history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=20
)

# Save the trained model
model.save('EfficientNetB7_Model_food_classifier_model.h5')